# 1. Data procurement
 
*Date: July 31, 2023*  
*Author: Alicia Larsen*     
*Institution: The Research Institute of Sweden (RISE)*   
*Contact: alicia.hh.larsen@gmail.com*   

This is the 2nd notebook of 7, in the series "RISE Wildfire Prediction Using Machine Learning"

References: This notebook is based on the procedures in the notebook found on this [link](https://github.com/ornldaac/modis_restservice_qc_filter_Python/blob/master/modis_restservice_qc_filter_Python.ipynb). This notebook can also be found in /initial-eda/data-procurement/reference-notebook/download-modis-data-example-notebook.ipynb, on github.com:larsenalicia/RISE-wildfire-prediction.git

##### Keywords: LST, LSR, Fire, MODIS, Python

## Overview
This notebook will explore the access to 3 MODIS data products: 
* the Land Surface Temperature/Emissivity (MOD11A2), 
* Surface Reflectance (MOD09A1), and 
* Thermal Anomalies/Fire (MOD14A2).

## Prerequisites: 

* Python 2 or 3   
* Libraries: requests, json, datetime, pandas, numpy, matplotlib
---

## Imports:

In [ ]:
# Imports
import requests
import json
import datetime
import pandas as pd
import pickle

## Global Variables:

In [ ]:
# Import the global variables from globals.global_vars, as well as the required functions
from globals.global_vars import url, header, coordinate_description, lat, lon, start_year, end_year, products, bands, time_intervals, above_below_left_right
from procerdures.a_procurement import print_globals_of_interest, get_dates, print_parameters_of_interest, data_retrival

# Variables in use 
print_globals_of_interest()

## Validating connection
Some products are not available. Therefore, check first if the products of interest are.

In [ ]:
# Check that every product is valid.
for product in products.values():
    response = requests.get(f'https://modis.ornl.gov/rst/api/v1/{product}/dates?latitude={lat}&longitude={lon}', headers=header)
    if str(response) == '<Response [200]>':
        print(f'{product} ok')
    else: 
        print(f'{product} not valid')

## Getting MODIS Dates 
For the scope of this predefined analysis, the time period 2018-2023 is chosen.    

Since all the products are located on the same satellite, it is only necesary to check the time period for one, and conclude that it is the same for all. The products are generated in 8-day composites for a total of 46 composite periods per calendar year.

The Modis date-system works as follows: 'A/year/day-of-year', meaning that the word-length is 8:

**MOD11A2** is generated in 8-day composites for a total of 46 composite periods per calendar year. The **MOD13Q1** has a 16-interval, and is hence the least frequently sampled product. This has a total of 23 composite periods per calendar year.

In [ ]:
# Define the number of datapoints per year, for 8 vs. 16 interval.
data_points_in_year_8 = 46
data_points_in_year_16 = 23

# Define the different time intervals
product_eg_8 = products['lst']
product_eg_16 = products['evi']

# Employ the get_dates() function to retrieve the dates of the time interval of interest.
dates_mod_8, dates_real_8 = get_dates(data_points_in_year_8, product_eg_8)
dates_mod_16, dates_real_16 = get_dates(data_points_in_year_16, product_eg_16)

# Save the lists to a file
with open('globals/most_frequent.ob', 'wb') as fp:
    pickle.dump(dates_real_8, fp)
with open('globals/least_frequent.ob', 'wb') as fp:
    pickle.dump(dates_real_16, fp)

## Request data

In [ ]:
# Input variables
print_parameters_of_interest()

In [ ]:
# Iterate through the dictionary of products
for key in products.keys():
    
    # Get the product code name
    product = products[key]

    # Iterate through the list of bands per product
    for band in bands[product]:
        
        # Get the data and write it to a CSV. Choose between the date-interval
        if time_intervals[product] == 8:
            data_retrival(key, band, dates_mod_8, dates_real_8)
        elif time_intervals[product] == 16:
            data_retrival(key, band, dates_mod_16, dates_real_16)
        else:
            raise KeyError('The dates for this time interval has not been calculated')

## Wrap-up
Now you should have all datasets to continue the analysis.

Have a nice day!

/ Alicia